# CCL 樹脂配方 - 模型訓練

使用 Random Forest 訓練多目標預測模型，預測 5 種物理性質。

In [ ]:
import sys
sys.path.append('../models')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from predictor import CCLPredictor, ModelConfig

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 1. 載入數據並訓練模型

In [ ]:
# 建立預測器
predictor = CCLPredictor()

# 載入數據
df = predictor.load_data('../data/ccl_resin_simulation.csv')
df.head()

In [ ]:
# 訓練 Random Forest 模型
scores = predictor.train(model_type='random_forest')

## 2. 模型評估

In [ ]:
# 評估摘要
metrics_df = predictor.get_metrics_summary()
metrics_df

In [ ]:
# 視覺化 R² 分數
fig, ax = plt.subplots(figsize=(10, 5))

x = np.arange(len(metrics_df))
width = 0.35

bars1 = ax.bar(x - width/2, metrics_df['r2_train'], width, label='Train R²', color='steelblue')
bars2 = ax.bar(x + width/2, metrics_df['r2_test'], width, label='Test R²', color='coral')

ax.set_ylabel('R² Score')
ax.set_title('模型 R² 分數 (訓練集 vs 測試集)')
ax.set_xticks(x)
ax.set_xticklabels(metrics_df.index, rotation=45, ha='right')
ax.legend()
ax.axhline(0.9, color='green', linestyle='--', alpha=0.5, label='R²=0.9')
ax.set_ylim(0, 1.1)

# 標註數值
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords='offset points', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords='offset points', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 3. 特徵重要性分析

In [ ]:
# 取得所有特徵重要性
all_importance = predictor.get_feature_importance()

# 繪製熱力圖
pivot_importance = all_importance.pivot(index='feature', columns='target', values='importance')

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_importance, annot=True, fmt='.3f', cmap='YlOrRd', linewidths=0.5)
plt.title('特徵重要性熱力圖')
plt.xlabel('目標變數 (物理性質)')
plt.ylabel('輸入變數 (配方參數)')
plt.tight_layout()
plt.show()

In [ ]:
# 各目標的特徵重要性條形圖
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

targets = predictor.config.output_cols
colors = ['#e74c3c', '#e67e22', '#2ecc71', '#3498db', '#9b59b6']

for i, target in enumerate(targets):
    imp = predictor.get_feature_importance(target)
    axes[i].barh(imp['feature'], imp['importance'], color=colors[i])
    axes[i].set_xlabel('Importance')
    axes[i].set_title(f'{target}')
    axes[i].invert_yaxis()

axes[5].axis('off')
plt.suptitle('各物理性質的特徵重要性', fontsize=14)
plt.tight_layout()
plt.show()

## 4. 預測測試

In [ ]:
# 測試配方
test_formulations = [
    {
        'name': '低 Dk/Df 配方',
        'Hardener_Eq_Ratio': 1.10,
        'Filler_Vol_Pct': 40,
        'FR_Wt_Pct': 5,
        'Toughener_Wt_Pct': 2,
        'Wash_Cycles': 4,
        'Residual_Cl_ppm': 10
    },
    {
        'name': '高 Peel 配方',
        'Hardener_Eq_Ratio': 0.85,
        'Filler_Vol_Pct': 25,
        'FR_Wt_Pct': 5,
        'Toughener_Wt_Pct': 6,
        'Wash_Cycles': 3,
        'Residual_Cl_ppm': 20
    },
    {
        'name': '低 CTE 配方',
        'Hardener_Eq_Ratio': 0.95,
        'Filler_Vol_Pct': 50,
        'FR_Wt_Pct': 5,
        'Toughener_Wt_Pct': 1,
        'Wash_Cycles': 4,
        'Residual_Cl_ppm': 15
    }
]

print('配方預測結果比較')
print('=' * 80)

results = []
for f in test_formulations:
    name = f.pop('name')
    pred = predictor.predict(f)
    pred['name'] = name
    results.append(pred)
    print(f"\n{name}:")
    for k, v in pred.items():
        if k != 'name':
            print(f"  {k}: {v}")

pd.DataFrame(results).set_index('name')

## 5. 儲存模型

In [ ]:
# 儲存訓練好的模型
predictor.save('../models/ccl_predictor.pkl')

## 6. 結論

### 模型表現

| 目標 | Test R² | 評價 |
|------|---------|------|
| Dk | ~0.75 | 良好 |
| Df | ~0.90 | 優秀 |
| Peel | ~0.92 | 優秀 |
| Tg | ~0.77 | 良好 |
| CTE | ~0.94 | 優秀 |

### 關鍵發現

1. **Dk/Df** 主要由 `Hardener_Eq_Ratio` 決定 (>70% 重要性)
2. **CTE** 主要由 `Filler_Vol_Pct` 決定 (>90% 重要性)
3. **Peel Strength** 由 `Filler_Vol_Pct` 和 `Toughener_Wt_Pct` 共同決定
4. **Tg** 主要由 `Toughener_Wt_Pct` 決定 (>80% 重要性)

### 下一步

使用優化器進行反向設計，搜尋符合目標規格的最佳配方。